# Trp-cage haMSM Construction and Analysis

For these examples, we'll be constructing an haMSM from simulations of Trp-cage unfolding.

In [1]:
from msm_we import msm_we
import numpy as np
import ray

The simulation data I'm using has already been augmented with pairwise alpha-carbon RMSDs, so we just need to strip out an extra dimension that's present in there.

(That extra dimension is just an artifact of how this data was prepared)

```
R = rms.RMSD(structure,  # universe to align
             structure,  # reference universe or atomgroup
             select='name CA',  # group to superimpose and calculate RMSD
             groupselections=['name CA'],  # groups for RMSD
             ref_frame=0)  # frame index of the reference
R.run()
```

This is just to clean up the logging output for display in the documentation webpage.

In [2]:
msm_we.log.handlers[0]._log_render.show_time = False
msm_we.log.handlers[0].console.width = 65

## Model building

First, let's set some parameters for haMSM building.

In [3]:
h5file_paths = ['data/west.h5']

# Number of MSM microstates to initially put in each stratum/WE bin
clusters_per_stratum = 35

dimreduce_method = 'vamp'

# Boundaries of the basis/target, in progress coordinate space
pcoord_bounds = {
    'basis': [[0, 0.1]],
    'target': [[0.7, 100]]
}

model_name = 'NaCl Sample'

# Reference structure
ref_file = 'data/2JOF.pdb'

# WESTPA resampling time
tau = 1e-9

In [4]:
def processCoordinates(self, coords):
    
    return coords.reshape(coords.shape[0], -1)
    
msm_we.modelWE.processCoordinates = processCoordinates

### With `build_analyze_model`

You can construct an haMSM (and validation haMSMs) with one call to `build_analyze_model()`.

In [5]:
model = msm_we.modelWE()

model.build_analyze_model(
    file_paths=h5file_paths,
    # ref_struct=basis_ref_dict,
    ref_struct=ref_file,
    modelName=model_name,
    basis_pcoord_bounds=pcoord_bounds['basis'],
    target_pcoord_bounds=pcoord_bounds['target'],
    dimreduce_method=dimreduce_method,
    n_clusters=clusters_per_stratum,
    tau=tau
)

Output()

2022-10-03 17:30:44,884 INFO worker.py:1509 -- Started a local 
Ray instance. View the dashboard at 127.0.0.1:8265 

Getting coordSet:   0%|          | 0/100 [00:00<?, ?it/s]

INFO     Performing weighted vamp                  ]8;id=634340;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=704428;file:///home/jd/msm_we/msm_we/msm_we.py#2769\2769]8;;\

INFO     Weighted vamp will reduce 816 to 496      ]8;id=862489;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=826572;file:///home/jd/msm_we/msm_we/msm_we.py#2782\2782]8;;\
         components.                                             

...

Submitting fluxmatrix tasks:   0%|          | 0/49 [00:00<?, ?it/s]

Retrieving flux matrices:   0%|          | 0/49 [00:00<?, ?it/s]

INFO     Fluxmatrices all obtained                 ]8;id=390345;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=511031;file:///home/jd/msm_we/msm_we/msm_we.py#4926\4926]8;;\

INFO                                               ]8;id=426819;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=389653;file:///home/jd/msm_we/msm_we/msm_we.py#5571\5571]8;;\
         Flux converged to 2.0039e+06 after 1                    
         iterations of inverse iteration.                        

INFO     Done with steady-state estimation.        ]8;id=668328;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=703718;file:///home/jd/msm_we/msm_we/msm_we.py#5582\5582]8;;\

### Step-by-step

`build_analyze_model()` is just a convenient wrapper around the following steps.

You can run them manually if you want to observe each step of the process.

It's helpful to start off by running step-by-step while you're starting analysis for a system, to fine-tune parameters without re-running the entire workflow.

In [6]:
ray.init(ignore_reinit_error=True)

2022-10-03 17:31:04,428 INFO worker.py:1351 -- Calling ray.init()
again after it has already been called.

Python version:,3.9.13
Ray version:,2.0.0
Dashboard:,http://127.0.0.1:8265


In [7]:
model = msm_we.modelWE()

In [9]:
model.initialize(
    fileSpecifier=h5file_paths,
    refPDBfile=ref_file,
    modelName=model_name,
    basis_pcoord_bounds=pcoord_bounds['basis'],
    target_pcoord_bounds=pcoord_bounds['target'],
    dim_reduce_method=dimreduce_method,
    tau=tau
)

In [10]:
model.get_iterations()
model.get_coordSet(last_iter = model.maxIter, streaming=True)

Getting coordSet:   0%|          | 0/100 [00:00<?, ?it/s]

In [11]:
model.dimReduce()

INFO     Performing weighted vamp                  ]8;id=712020;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=720830;file:///home/jd/msm_we/msm_we/msm_we.py#2769\2769]8;;\

INFO     Weighted vamp will reduce 816 to 496      ]8;id=994593;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=859177;file:///home/jd/msm_we/msm_we/msm_we.py#2782\2782]8;;\
         components.                                             

In [13]:
model.cluster_coordinates(
    n_clusters=clusters_per_stratum,
    use_ray=True,
    stratified=True,
    store_validation_model=True # Required for block validation
)

INFO     Be aware: Number of cluster centers is an ]8;id=929913;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=855513;file:///home/jd/msm_we/msm_we/msm_we.py#3057\3057]8;;\
         important parameter, and can drastically                
         affect model quality. We recommend                      
         examining block-validation results with a               
         range of numbers of clusters, to check                  
         for overfitting.                                        

INFO     Beginning stratified clustering.          ]8;id=920873;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=109884;file:///home/jd/msm_we/msm_we/msm_we.py#3064\3064]8;;\

INFO     Obtaining bin definitions from iteration  ]8;id=322883;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=10176;file:///home/jd/msm_we/msm_we/msm_we.py#3482\3482]8;;\
         2 in file data/west.h5                                  

INFO     Loading pickled bin mapper from H5 for    ]8;id=893816;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=116602;file:///home/jd/msm_we/msm_we/msm_we.py#3486\3486]8;;\
         stratified clustering...                                

ERROR    Stratified clustering currently MUST run  ]8;id=113502;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=650222;file:///home/jd/msm_we/msm_we/msm_we.py#3522\3522]8;;\
         in streaming mode (and with ray enabled                 
         for discretization). Enabling both and                  
         continuing...                                           

...

INFO     Clustering couldn't use last iteration,   ]8;id=539257;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=160445;file:///home/jd/msm_we/msm_we/msm_we.py#3595\3595]8;;\
         not all bins filled.                                    

INFO     Using Ray cluster with 16.0 CPUs!          ]8;id=426758;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=376163;file:///home/jd/msm_we/msm_we/msm_we.py#873\873]8;;\

Submitting discretization tasks:   0%|          | 0/99 [00:00<?, ?it/s]

Retrieving discretized trajectories:   0%|          | 0/99 [00:00<?, ?it/s]

INFO     Discretization complete                   ]8;id=860343;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=741961;file:///home/jd/msm_we/msm_we/msm_we.py#4177\4177]8;;\

In [14]:
model.get_fluxMatrix(n_lag=0)

Constructing flux matrix:   0%|          | 0/98 [00:00<?, ?it/s]

In [15]:
model.organize_fluxMatrix()

INFO     Started with 420 clusters, and removed    ]8;id=439948;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=832126;file:///home/jd/msm_we/msm_we/msm_we.py#4001\4001]8;;\
         350                                                     

WARNING  All clusters were cleaned from bins {11}  ]8;id=942130;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=335258;file:///home/jd/msm_we/msm_we/msm_we.py#4017\4017]8;;\

INFO     Using Ray cluster with 16.0 CPUs!          ]8;id=366468;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=712475;file:///home/jd/msm_we/msm_we/msm_we.py#873\873]8;;\

INFO     Using cached model for discretization     ]8;id=365307;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=872011;file:///home/jd/msm_we/msm_we/msm_we.py#4113\4113]8;;\

Submitting discretization tasks:   0%|          | 0/99 [00:00<?, ?it/s]

Retrieving discretized trajectories:   0%|          | 0/99 [00:00<?, ?it/s]

INFO     Discretization complete                   ]8;id=718435;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=758906;file:///home/jd/msm_we/msm_we/msm_we.py#4177\4177]8;;\

/home/jd/msm_we/msm_we/msm_we.py:4248: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.targetRMSD_all = np.array(cluster_pcoord_all)[pcoord_sort_indices]


Constructing flux matrix:   0%|          | 0/98 [00:00<?, ?it/s]

In [16]:
model.get_Tmatrix()

In [17]:
model.get_steady_state()

INFO                                               ]8;id=299096;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=28391;file:///home/jd/msm_we/msm_we/msm_we.py#5571\5571]8;;\
         Flux converged to 1.2810e+06 after 1                    
         iterations of inverse iteration.                        

INFO     Done with steady-state estimation.        ]8;id=939558;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=655330;file:///home/jd/msm_we/msm_we/msm_we.py#5582\5582]8;;\

In [18]:
model.get_steady_state_target_flux()

In [19]:
model.do_block_validation(
    cross_validation_groups=2, 
    cross_validation_blocks=4
)

INFO     Beginning analysis of cross-validation    ]8;id=167282;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=140219;file:///home/jd/msm_we/msm_we/msm_we.py#1322\1322]8;;\
         group 1/2.                                              

INFO     Using Ray cluster with 16.0 CPUs!          ]8;id=916799;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=79005;file:///home/jd/msm_we/msm_we/msm_we.py#873\873]8;;\

Submitting fluxmatrix tasks:   0%|          | 0/50 [00:00<?, ?it/s]

Retrieving flux matrices:   0%|          | 0/50 [00:00<?, ?it/s]

INFO     Fluxmatrices all obtained                 ]8;id=331668;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=15542;file:///home/jd/msm_we/msm_we/msm_we.py#4926\4926]8;;\

...

Submitting fluxmatrix tasks:   0%|          | 0/49 [00:00<?, ?it/s]

Retrieving flux matrices:   0%|          | 0/49 [00:00<?, ?it/s]

INFO     Fluxmatrices all obtained                 ]8;id=376616;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=857381;file:///home/jd/msm_we/msm_we/msm_we.py#4926\4926]8;;\

INFO                                               ]8;id=88839;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=868699;file:///home/jd/msm_we/msm_we/msm_we.py#5571\5571]8;;\
         Flux converged to 2.0039e+06 after 1                    
         iterations of inverse iteration.                        

INFO     Done with steady-state estimation.        ]8;id=773865;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=837127;file:///home/jd/msm_we/msm_we/msm_we.py#5582\5582]8;;\

## Save model

In [20]:
import pickle
with open('data/pickled_model', 'wb') as of:
    pickle.dump(model, of)